# Lesson 4: Persistence and Streaming

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [6]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [8]:
messages = [HumanMessage(content="What is the weather in Chiniot?")]

In [9]:
thread = {"configurable": {"thread_id": "1"}}

In [10]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WCyu4S4SqcyvNJqw9b3TWwc5', 'function': {'arguments': '{"query":"current weather in Chiniot"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-da30cf30-4a59-4272-944f-e275b4831e19-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Chiniot'}, 'id': 'call_WCyu4S4SqcyvNJqw9b3TWwc5'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Chiniot'}, 'id': 'call_WCyu4S4SqcyvNJqw9b3TWwc5'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Chiniot\', \'region\': \'Punjab\', \'country\': \'Pakistan\', \'lat\': 31.72, \'lon\': 72.98, \'tz_id\

In [11]:
messages = [HumanMessage(content="What about in Faisalabad?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_B7mIsqlYWEFLFZb14FBmSTuJ', 'function': {'arguments': '{"query":"current weather in Faisalabad"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 843, 'total_tokens': 866}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ea592b34-4d2d-401f-8b4d-ac59dacad953-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Faisalabad'}, 'id': 'call_B7mIsqlYWEFLFZb14FBmSTuJ'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Faisalabad'}, 'id': 'call_B7mIsqlYWEFLFZb14FBmSTuJ'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Faisalabad\', \'region\': \'Punjab\', \'country\': \'Pakistan\', \

In [12]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Both Chiniot and Faisalabad have similar temperatures, but Chiniot is slightly warmer. \n\n- **Chiniot**: 32.1°C (89.9°F), Feels Like 38.0°C (100.4°F)\n- **Faisalabad**: 31.8°C (89.3°F), Feels Like 37.6°C (99.8°F)\n\nSo, Chiniot is marginally warmer than Faisalabad.', response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 1537, 'total_tokens': 1638}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f27e90a-e64b-41bc-98b3-12ad3aabe6d5-0')]}


In [13]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please specify the two locations, objects, or time periods you are comparing in terms of warmth? This will help me provide a more accurate answer.', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 149, 'total_tokens': 181}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea94d217-b1ad-4654-bae8-9738122b4645-0')]}


## Streaming tokens

In [14]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [16]:
messages = [HumanMessage(content="What is the weather in Layyah?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="")

The current weather in Layyah, Pakistan is as follows:

- **Temperature:** 32.9°C (91.2°F)
- **Condition:** Clear
- **Wind:** 10.7 mph (17.3 kph) from the ENE
- **Pressure:** 994.0 mb
- **Humidity:** 78%
- **Visibility:** 10 km
- **Feels like:** 45.5°C (113.9°F)

It's important to check local weather updates for the most accurate and current information.